# Data Transformation
"Documentation for the ETL process, including data sources and transformation logic."

In [1]:
import pandas as pd

### Loading the cleaned data source for Global Finance

In [2]:
df = pd.read_csv("global_finance_data_cleaned.csv")
df.head(5) #check first 5 records to see if it has loaded correctly

,Stock_Index,Country,Currency_Code,Index_Value,Daily_Change_Percent,Market_Cap_Trillion_USD,GDP_Growth_Rate_Percent,Inflation_Rate_Percent,Interest_Rate_Percent,Unemployment_Rate_Percent,...,Current_Account_Balance_Billion_USD,FDI_Inflow_Billion_USD,Commodity_Index,Bond_Yield_10Y_Percent,Credit_Rating,Political_Risk_Score,Banking_Sector_Health,Real_Estate_Index,Export_Growth_Percent,Import_Growth_Percent
0,S&P_500,United States,USD,5437.2,0.34,51.2,2.8,2.9,5.50,3.7,...,-695.2,456.8,1.12,4.25,AAA,8.1,Strong,145.6,3.2,2.8
1,Shanghai_Composite,China,CNY,2891.6,-0.82,12.4,5.2,0.8,3.10,5.2,...,382.9,189.7,0.98,2.15,A+,6.7,Moderate,98.7,8.9,6.1
2,Nikkei_225,Japan,JPY,36789.1,1.24,6.8,0.9,2.8,-0.10,2.4,...,49.7,23.4,1.05,0.89,A+,8.4,Strong,89.3,5.1,4.7
3,DAX,Germany,EUR,18234.5,0.67,2.9,0.3,2.2,4.50,3.1,...,297.4,67.8,1.08,2.31,AAA,8.7,Strong,112.4,2.1,1.8
4,FTSE_100,United Kingdom,GBP,8156.3,-0.15,3.1,1.1,2.0,5.25,4.2,...,-85.6,45.2,1.06,3.89,AA,7.9,Moderate,97.8,0.9,1.2


### Best Practice - making a dataframe copy of the original clean dataset to work with 

In [3]:
gfd_df = df.copy() #best practice

### Setting the Multi-Index (using unique values)

In [4]:
gfd_df = gfd_df.set_index(['Stock_Index','Currency_Code']) #multi-indexing
gfd_df.index

MultiIndex([(           'S&P_500', 'USD'),
            ('Shanghai_Composite', 'CNY'),
            (        'Nikkei_225', 'JPY'),
            (               'DAX', 'EUR'),
            (          'FTSE_100', 'GBP'),
            (            'CAC_40', 'EUR'),
            (            'Sensex', 'INR'),
            (               'TSX', 'CAD'),
            (           'Bovespa', 'BRL'),
            (           'ASX_200', 'AUD'),
            (             'KOSPI', 'KRW'),
            (              'MOEX', 'RUB'),
            (               'IPC', 'MXN'),
            (          'FTSE_MIB', 'EUR'),
            (           'IBEX_35', 'EUR'),
            (               'AEX', 'EUR'),
            (               'SMI', 'CHF'),
            (     'OMX_Stockholm', 'SEK'),
            (               'OSE', 'NOK'),
            (           'OMXC_20', 'DKK'),
            (               'STI', 'SGD'),
            (         'Hang_Seng', 'HKD'),
            (             'TAIEX', 'TWD'),
           

### Best Practice - changing data types to Category where appropriate

In [5]:
#Best Practice - using the category types
gfd_df['Credit_Rating'] = gfd_df['Credit_Rating'].astype('category')
gfd_df['Banking_Sector_Health'] = gfd_df['Banking_Sector_Health'].astype('category')
gfd_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 39 entries, ('S&P_500', 'USD') to ('TA_125', 'ILS')
Data columns (total 21 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   Country                              39 non-null     object  
 1   Index_Value                          39 non-null     float64 
 2   Daily_Change_Percent                 39 non-null     float64 
 3   Market_Cap_Trillion_USD              39 non-null     float64 
 4   GDP_Growth_Rate_Percent              39 non-null     float64 
 5   Inflation_Rate_Percent               39 non-null     float64 
 6   Interest_Rate_Percent                39 non-null     float64 
 7   Unemployment_Rate_Percent            39 non-null     float64 
 8   Exchange_Rate_USD                    39 non-null     float64 
 9   Currency_Change_YTD_Percent          39 non-null     float64 
 10  Government_Debt_GDP_Percent          39 non-null     flo

## Feature Engineering & Transformation Logic:

### Defining the Function for new Feature: Country_Group

In [6]:
def find_country_group(country):

    country_groups = {
    
    ('United States','Japan','Germany','United Kingdom','France','Canada','Italy'):'G7',
    ('China','India','Brazil','Russia','Hong Kong','Indonesia','South Africa','Egypt','UAE','Saudi Arabia'):'BRICS',
    ('Australia','South Korea','Mexico','Switzerland','Turkey','Argentina'):'G20',
    ('Nigeria'):'Africa',
    ('Singapore','Taiwan','Thailand','Malaysia','Philippines','Vietnam'):'Asia',
    ('Spain','Netherlands','Sweden','Norway','Denmark'):'Europe',
    ('Israel'):'Middle East',
    ('Chile','Colombia','Peru'):'South America'
    
}
    for k_country, v_group in country_groups.items():
      if country in k_country:
          return v_group               

### Applying the function to the dataframe using Lambda - then changing the data type of new feature to a category

In [7]:
gfd_df['Country_Group_Derived'] = gfd_df.apply(lambda row: find_country_group(row['Country']), axis=1)
gfd_df['Country_Group_Derived'] = gfd_df['Country_Group_Derived'].astype('category') #Best Practice - using the category types
gfd_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 39 entries, ('S&P_500', 'USD') to ('TA_125', 'ILS')
Data columns (total 22 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   Country                              39 non-null     object  
 1   Index_Value                          39 non-null     float64 
 2   Daily_Change_Percent                 39 non-null     float64 
 3   Market_Cap_Trillion_USD              39 non-null     float64 
 4   GDP_Growth_Rate_Percent              39 non-null     float64 
 5   Inflation_Rate_Percent               39 non-null     float64 
 6   Interest_Rate_Percent                39 non-null     float64 
 7   Unemployment_Rate_Percent            39 non-null     float64 
 8   Exchange_Rate_USD                    39 non-null     float64 
 9   Currency_Change_YTD_Percent          39 non-null     float64 
 10  Government_Debt_GDP_Percent          39 non-null     flo

### Defining the Function for 2 new Features: Credit Grade Derived and Credit Grade Classification Derived
Using 1 function to derive 2 values, depending if there is a second value passed to it (2nd value defaults to 0)

In [8]:
def snpCRlookup(credit_rating,info=0):
        
        ratings = {
            
            ('AAA'):['Prime', 'Investment'],
            ('AA+','AA','AA-'):['High Grade', 'Investment'],
            ('A+','A','A-'):['Upper Medium Grade', 'Investment'],
            ('BBB+','BBB','BBB-'):['Lower Medium Grade','Investment'],
            ('BB+','BB','BB-'):['Speculative', 'Non-Investment/Speculative'],
            ('B+','B','B-'):['Highly Speculative','Non-Investment/Speculative'],
            ('CCC+','CCC','CCC-'):['Substantial Risk','Non-Investment/Speculative'],
            ('CC'):['Extremely Speculative','Non-Investment/Speculative'],
            ('C'):['Near Default','Non-Investment/Speculative'],
            ('D'):['Default','Non-Investment/Speculative'],
            ('R'):['Regulatory Supervision','Special Classification'],
            ('SD'):['Speculative Default','Special Classification'],
            ('NR'):['Not Rated','Special Classification']
        
        }
    
        for key, value in ratings.items():
            
                if credit_rating in key:
                    
                    if info == 0:
                        return value[0]
                        
                    else:
                        return value[1]


### Applying the function to the dataframe using Lambda - then changing the data type of new features to a category

In [9]:
gfd_df['Credit_Grade_Derived'] = gfd_df.apply(lambda row: snpCRlookup(row['Credit_Rating']), axis=1)
gfd_df['CR_Grade_Classification_Derived'] = gfd_df.apply(lambda row: snpCRlookup(row['Credit_Rating'],1), axis=1)
gfd_df['Credit_Grade_Derived'] = gfd_df['Credit_Grade_Derived'].astype('category')
gfd_df['CR_Grade_Classification_Derived'] = gfd_df['CR_Grade_Classification_Derived'].astype('category')
gfd_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 39 entries, ('S&P_500', 'USD') to ('TA_125', 'ILS')
Data columns (total 24 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   Country                              39 non-null     object  
 1   Index_Value                          39 non-null     float64 
 2   Daily_Change_Percent                 39 non-null     float64 
 3   Market_Cap_Trillion_USD              39 non-null     float64 
 4   GDP_Growth_Rate_Percent              39 non-null     float64 
 5   Inflation_Rate_Percent               39 non-null     float64 
 6   Interest_Rate_Percent                39 non-null     float64 
 7   Unemployment_Rate_Percent            39 non-null     float64 
 8   Exchange_Rate_USD                    39 non-null     float64 
 9   Currency_Change_YTD_Percent          39 non-null     float64 
 10  Government_Debt_GDP_Percent          39 non-null     flo

### Defining the Function for new Feature: Banking Sector Health Number (Derived)
Note to self: Will do this using pipes when considering Machine Learning

In [10]:
def convert_bankingsechealth_tonum(banking_sector_health):

    bsh_mappings = {
    
    'Strong': 1,
    'Moderate': 2,
    'Weak': 3,
    
}
    return bsh_mappings[banking_sector_health]               

### Applying the function to the dataframe using Lambda - then changing the data type of new feature to a category

In [11]:
gfd_df['Banking_Sector_Health_Num_Derived'] = gfd_df.apply(lambda row: convert_bankingsechealth_tonum(row['Banking_Sector_Health']), axis=1)
gfd_df['Banking_Sector_Health_Num_Derived'] = gfd_df['Banking_Sector_Health_Num_Derived'].astype('category')
gfd_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 39 entries, ('S&P_500', 'USD') to ('TA_125', 'ILS')
Data columns (total 25 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   Country                              39 non-null     object  
 1   Index_Value                          39 non-null     float64 
 2   Daily_Change_Percent                 39 non-null     float64 
 3   Market_Cap_Trillion_USD              39 non-null     float64 
 4   GDP_Growth_Rate_Percent              39 non-null     float64 
 5   Inflation_Rate_Percent               39 non-null     float64 
 6   Interest_Rate_Percent                39 non-null     float64 
 7   Unemployment_Rate_Percent            39 non-null     float64 
 8   Exchange_Rate_USD                    39 non-null     float64 
 9   Currency_Change_YTD_Percent          39 non-null     float64 
 10  Government_Debt_GDP_Percent          39 non-null     flo

### Checking the new Banking Sector Health numbers are correct

In [12]:
gfd_df[['Banking_Sector_Health',
        'Banking_Sector_Health_Num_Derived']]

,,Banking_Sector_Health,Banking_Sector_Health_Num_Derived
Stock_Index,Currency_Code,,
S&P_500,USD,Strong,1
Shanghai_Composite,CNY,Moderate,2
Nikkei_225,JPY,Strong,1
DAX,EUR,Strong,1
FTSE_100,GBP,Moderate,2
CAC_40,EUR,Moderate,2
Sensex,INR,Moderate,2
TSX,CAD,Strong,1
Bovespa,BRL,Weak,3


### Building in a suitable proxy for country's GDP value and 5 new ratio features missing from the original dataset
1. A GDP Proxy (USD Billions) created by converting Market Capital (USD Trillion) into USD Billion
2. Current Account to GDP Ratio (%age)
3. Export-to-Import Growth Ratio
4. FDI-to-GDP Ratio (%age)
5. Interest Rate Spread
6. Unemployment-to-GDP Growth Ratio (%age)

In [13]:
#A GDP Proxy (USD Billions) created by converting Market Capital (USD Trillion) into USD Billion
gfd_df['GDP_Proxy_Billion_USD'] = gfd_df['Market_Cap_Trillion_USD'] * 1000

#Current Account to GDP Ratio (%age)
gfd_df['Current_Account_To_GDP_Ratio'] = (gfd_df['Current_Account_Balance_Billion_USD']/gfd_df['GDP_Proxy_Billion_USD'])*100

#Export-to-Import Growth Ratio
gfd_df['Export_To_Import_Growth_Ratio'] = gfd_df['Export_Growth_Percent']/(gfd_df['Import_Growth_Percent'])

#FDI-to-GDP Ratio (%age)
gfd_df['FDI_To_GDP_Ratio'] = (gfd_df['FDI_Inflow_Billion_USD']/gfd_df['GDP_Proxy_Billion_USD'])*100

#Interest Rate Spread
gfd_df['Interest_Rate_Spread'] = gfd_df['Interest_Rate_Percent']-gfd_df['Bond_Yield_10Y_Percent']

#Unemployment-to-GDP Growth Ratio
gfd_df['Unemployment_To_GDP_Growth_Ratio'] = (gfd_df['Unemployment_Rate_Percent']/gfd_df['GDP_Proxy_Billion_USD'])*100

In [14]:
gfd_df.head(3)

,,Country,Index_Value,Daily_Change_Percent,Market_Cap_Trillion_USD,GDP_Growth_Rate_Percent,Inflation_Rate_Percent,Interest_Rate_Percent,Unemployment_Rate_Percent,Exchange_Rate_USD,Currency_Change_YTD_Percent,...,Country_Group_Derived,Credit_Grade_Derived,CR_Grade_Classification_Derived,Banking_Sector_Health_Num_Derived,GDP_Proxy_Billion_USD,Current_Account_To_GDP_Ratio,Export_To_Import_Growth_Ratio,FDI_To_GDP_Ratio,Interest_Rate_Spread,Unemployment_To_GDP_Growth_Ratio
Stock_Index,Currency_Code,,,,,,,,,,,,,,,,,,,,,
S&P_500,USD,United States,5437.2,0.34,51.2,2.8,2.9,5.5,3.7,1.00,0.0,...,G7,Prime,Investment,1,51200.0,-1.357813,1.142857,0.892188,1.25,0.007227
Shanghai_Composite,CNY,China,2891.6,-0.82,12.4,5.2,0.8,3.1,5.2,7.28,2.3,...,BRICS,Upper Medium Grade,Investment,2,12400.0,3.087903,1.459016,1.529839,0.95,0.041935
Nikkei_225,JPY,Japan,36789.1,1.24,6.8,0.9,2.8,-0.1,2.4,147.20,-8.9,...,G7,Upper Medium Grade,Investment,1,6800.0,0.730882,1.085106,0.344118,-0.99,0.035294


In [15]:
gfd_df.info() #a final check to see if all the new features have been added to the dataframe before saving as new file

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 39 entries, ('S&P_500', 'USD') to ('TA_125', 'ILS')
Data columns (total 31 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   Country                              39 non-null     object  
 1   Index_Value                          39 non-null     float64 
 2   Daily_Change_Percent                 39 non-null     float64 
 3   Market_Cap_Trillion_USD              39 non-null     float64 
 4   GDP_Growth_Rate_Percent              39 non-null     float64 
 5   Inflation_Rate_Percent               39 non-null     float64 
 6   Interest_Rate_Percent                39 non-null     float64 
 7   Unemployment_Rate_Percent            39 non-null     float64 
 8   Exchange_Rate_USD                    39 non-null     float64 
 9   Currency_Change_YTD_Percent          39 non-null     float64 
 10  Government_Debt_GDP_Percent          39 non-null     flo

### Finaly, saving the dataframe again (with indexed columns) for further analysis

In [16]:
gfd_df.to_csv('global_finance_data_FeatureEng.csv', index=True)